In [86]:
import pandas as pd
import numpy as np
import datetime
s = "C:\\Users\\Matvey Solovyov\\OneDrive\\Documents\\UWA\\2021_S1\\GinginWx\\gingin-weather\gingin_aero1.csv"

In [23]:
df = pd.read_csv(s, index_col='OBSID', usecols=['OBSID','LSD','PRCP_9AM','PRCP_10','MSG'])

df['LSD'] = pd.to_datetime( df['LSD'] )

In [505]:
def make_prcp_interval(df):
    # drop any columns where PRCP columns are NaNs.
    
    # can have at maximum double the number of rows, if every row has significant 10min precip
    df_new = pd.DataFrame(columns=['date','prcp'], index = range(1,2*len(df)))

    df_new.iloc[0][['date','prcp']] = df.iloc[0][['LSD', 'PRCP_10']]

    # define an index to keep track of check_new
    j = 1

    # loop through check
    for i in range(1,len(df)):

        # If it's the first row after 9am, will be handled differently.
        
        if df.iloc[i]['LSD'].time() > datetime.time(9,0):
            # if current row is after 9 am, check if previous is after 9 and on the same day
            previous_after_9AM = df.iloc[i - 1]['LSD'].date() == df.iloc[i]['LSD'].date() and df.iloc[i - 1]['LSD'].time() > datetime.time(9,0) 
        else:
            # if current row is before 9 am, check if previous row is same day, or previous day but after 9 am on that day.
            previous_after_9AM = df.iloc[i - 1]['LSD'].date() == df.iloc[i]['LSD'].date() or \
                                        (df.iloc[i - 1]['LSD'].date() == df.iloc[i]['LSD'].date() - datetime.timedelta(days= 1) and \
                                         df.iloc[i - 1]['LSD'].time() > datetime.time(9,0))
                
                
        # have some prcp in the previous 10 mins and the previous record was more than 10 mins ago
        if df.iloc[i]['PRCP_10'] > 0 and (df.iloc[i]['LSD'] - df.iloc[i-1]['LSD']).seconds > 600:

            # add one row for precip fallen more than 10 mins before current row but after previous
            new_prcp_10 = df.iloc[i]['PRCP_9AM'] - df.iloc[i]['PRCP_10']

            # if previous record was not 9am, need to subtract previous precip
            if previous_after_9AM:

                new_prcp_10 -= df.iloc[i-1]['PRCP_9AM']

            df_new.iloc[j][['date','prcp']] = (df.iloc[i]['LSD'] - datetime.timedelta(minutes = 10) , abs(round(new_prcp_10,2)) )

            j += 1

            # add a second row for precip fallen in the 10 mins before this row
            df_new.iloc[j][['date','prcp']] = (df.iloc[i]['LSD'],
                                                  df.iloc[i]['PRCP_10'])

            j += 1

        # don't have precip in last 10 mins, or have, but previous record was 10 or less mins ago
        else:

            # set precip value to difference in 9am precip values
            # unless previous record was 9am, then don't need to subtract previous precip

            new_prcp = df.iloc[i]['PRCP_9AM']

            if previous_after_9AM:
                new_prcp -= df.iloc[i - 1]['PRCP_9AM']

            df_new.iloc[j][['date','prcp']] = (df.iloc[i]['LSD'], abs( round( new_prcp , 2 ) ) )
            j += 1


    return df_new.dropna()

In [506]:
metar_data = pd.DataFrame()

for i in range(1,4):
    s = "C:\\Users\\Matvey Solovyov\\OneDrive\\Documents\\UWA\\2021_S1\\GinginWx\\gingin-weather\gingin_aero"+str(i)+".csv"
    
    subdf = pd.read_csv(s, usecols=['LSD','PRCP_9AM','PRCP_10','MSG'] )
    
    metar_data = pd.concat([metar_data, subdf])
    
metar_data['LSD'] = pd.to_datetime( metar_data['LSD'] )
metar_data = metar_data.dropna()
    
print(len(metar_data))

171197


In [507]:
metar_prcp = make_prcp_interval(metar_data)

In [496]:
metar_prcp

,date,prcp
1,2007-08-21 14:00:00,0
2,2007-08-21 15:00:00,0
3,2007-08-21 16:00:00,0
4,2007-08-21 17:00:00,0
5,2007-08-21 18:00:00,0
...,...,...
175490,2020-11-10 08:49:59,0.4
175491,2020-11-10 08:55:00,0.8
175492,2020-11-10 09:00:00,0
175493,2020-11-10 09:30:00,0


In [497]:
max(metar_prcp['prcp'])

27.8

In [499]:
metar_prcp.iloc[32875:32885]

,date,prcp
32876,2011-02-16 15:54:00,0.4
32877,2011-02-16 16:00:00,1
32878,2011-02-16 16:50:00,27.8
32879,2011-02-16 17:00:00,3.6
32880,2011-02-16 17:50:00,2.6
32881,2011-02-16 18:00:00,0.2
32882,2011-02-16 19:00:00,0.6
32883,2011-02-16 20:00:00,0
32884,2011-02-16 21:00:00,0
32885,2011-02-16 22:00:00,0


In [502]:
metar_data[ (metar_data['LSD'].dt.day == 16) & (metar_data['LSD'].dt.month == 2) & (metar_data['LSD'].dt.year == 2011)]

,LSD,PRCP_9AM,PRCP_10,MSG
32229,2011-02-16 00:00:00,0.0,0.0,METARAWS YGIG 151600 31004/05KT //// 23.4/14.4...
32230,2011-02-16 01:00:00,0.0,0.0,METARAWS YGIG 151700 36003/04KT //// 23.4/16.7...
32231,2011-02-16 02:00:00,0.0,0.0,METARAWS YGIG 151800 00000/00KT //// 22.8/14.6...
32232,2011-02-16 03:00:00,0.0,0.0,METARAWS YGIG 151900 11010/14KT //// 26.7/15.4...
32233,2011-02-16 04:00:00,0.0,0.0,METARAWS YGIG 152000 09010/14KT //// 26.2/15.6...
32234,2011-02-16 05:00:00,0.0,0.0,METARAWS YGIG 152100 11009/12KT //// 25.7/16.0...
32235,2011-02-16 06:00:00,0.0,0.0,METARAWS YGIG 152200 10011/14KT //// 25.3/15.9...
32236,2011-02-16 07:00:00,0.0,0.0,METARAWS YGIG 152300 11011/14KT //// 26.5/16.4...
32237,2011-02-16 08:00:00,0.0,0.0,METARAWS YGIG 160000 09011/15KT //// 28.8/16.7...
32238,2011-02-16 09:00:00,0.0,0.0,METARAWS YGIG 160100 06010/13KT //// 31.6/16.7...


In [508]:
metar_prcp.to_csv("C:\\Users\\Matvey Solovyov\\OneDrive\\Documents\\UWA\\2021_S1\\GinginWx\\gingin-weather\gingin_metar_prcpINT.csv", index=False)